In [1]:
import requests
import json
import re
from bs4 import BeautifulSoup
import time
import pymongo
import json
import numpy as np
from matplotlib import pyplot as plt 
import pandas as pd
import csv

In [2]:
#--------- 启动 mongodb 服务 ---------#
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
# mydb = myclient["jdtest"]
mydb = myclient["kongtiao"]
# mycol = mydb["sites0602"]
mycol = mydb["sites06012"]
#--------- 启动 mongodb 服务 ---------#

In [3]:

def good_in_list(res,target,label,top):
    i = 0
    for x in res:
        re_arr = re.findall(target,x[label])
        
        if len(re_arr) > 0 and i <= top :
            return True
        i = i + 1
        if i>top:
            return False
    
def drawpandas(res):
    index = 0
    key_dict = {"综合系数":[],"地址":[],"价格":[],"制冷剂":[],"制冷功率(W)":[],"制热功率(W)":[],"能效等级":[],"产品匹数":[],"变频/定频":[]}
    res_tmp =[]
    
    for x in res:
        print((x))
        index = index + 1
        check = 0
        
        for i in key_dict:
            if (i in x) == False:
                x[i] = 'None'
            key_dict[i].append(x[i])
        res_tmp.append(x)
        
    return pd.DataFrame(key_dict)

In [16]:
res = mycol.find({},{"_id":0})
nx_map = {'3级':'三级能效','2级':'二级能效','1级':'新一级能效'}
i = 0
for x in res:
    i = i + 1
    myquery = { "地址": x["地址"] }
    if "能效等级" in x:
        print(x["能效等级"])
        if x["能效等级"] in nx_map:
            x["能效等级"] = nx_map[x["能效等级"]]
        newvalues = { "$set": {"能效等级": x["能效等级"]}}
        mycol.update_one(myquery, newvalues)

1级
1级
1级
3级
1级
3级
3级
3级
3级
2级
1级
3级
3级
3级
3级
2级
1级
3级
2级
1级
3级
3级
3级
2级
1级
3级
3级
3级
1级
1级
3级
2级
3级
2级
1级
1级
3级
3级
1级
1级
3级
2级
1级
1级
1级
2级
3级
1级
3级
1级
1级
3级
3级
1级
1级
3级
1级
3级
1级
1级
1级
1级
1级
3级
2级
3级
1级
3级
3级
1级
2级
1级
3级
3级
2级
1级
3级
1级
3级
3级
1级
1级
1级
3级
1级
2级
1级
1级
1级
1级
1级
2级
3级
1级
3级
1级
3级
1级
3级
3级
1级
1级
1级
1级
1级
1级
1级
1级
3级
1级
1级
1级
1级
1级
1级
1级
3级
2级
1级
1级
3级
1级
2级
2级
1级
3级
1级
1级
1级
1级
1级
1级
1级
1级
1级
1级
1级
1级
3级
1级
1级
1级
2级
2级
1级
3级
1级
1级
1级
1级
1级
3级
1级
1级
1级
1级
1级
2级
1级
1级
1级
1级
1级
1级
1级
1级
1级
3级
1级
1级
1级
1级
1级
1级
1级
1级
1级
1级
1级
1级
1级
2级
1级
3级
2级
1级
3级
1级
1级
2级
2级
1级
2级
3级
1级
2级
1级
1级
1级
3级
1级
1级
1级
1级
1级
1级
1级
1级
1级
1级
1级
1级
1级


In [29]:
res = mycol.find({},{"_id":0}).sort("价格")
i = 0
for x in res:    
#     print(x["名字"])
    tmp = x["名字"].replace('（','(').replace('）',')').replace('/','').replace('-','').replace('(WIFI)','').replace('(wifi)','').replace('(TM)','').replace('(JD)','')
#     print(tmp)
    myquery = { "地址": x["地址"] }
    newvalues = { "$set": { "名字": tmp } }
    if "产品型号" in x:
        newvalues["$set"]["产品型号"] = x["产品型号"].replace('（','(').replace('）',')').replace('/','').replace('-','').strip()
    mycol.update_one(myquery, newvalues)

## 获取所有格力商品条目

In [5]:
tmp_Str = 'https://mall.gree.com/'
res = mycol.find({"$or":[{'地址':{ "$regex": tmp_Str}}]},{"_id":0}).sort("价格")
res_gree = []
for x in res:
    patten = '([A-Za-z0-9]+\(?[A-Za-z0-9]*\)?[A-Za-z0-9]+)'
#     print(x["名字"])
    maxs = ""
    for m in re.findall(patten,x["名字"]):
        if len(maxs) < len(m):
            maxs = m
    res_gree.append(maxs)        
    print(maxs)

KFR26GWNhDaB3W
KFR26GW(26530)FNhAkB3
KFR35GWNhDdB3
KFR35GW(35530)FNhAkB3
KFR26GW(26530)FNhAkB1
KFR35GW(35530)FNhAkB1
KFR26GWNhGc3B
KFR26GWNhGc1B
KFR26GWNhBb3Bj
KFR26GWNhBa3Bt
KFR35GWNhGe3B
KFR26GW(26592)FNhAfB3
KYR35NANA1A
KFR35GWNhGc1B
KFR35GWNhBb3Bj
KFR35GWNhBa3Bt
KFR35GW(35592)FNhAfB3
KFR26GW(26563)FNhAaB3JY01
KFR26GW(26526)FNhAaB3
KFR35GW(35526)FNhAaB3
KFR35GW(35563)FNhAeB3JY01
KFR35GW(35563)FNhAaB2JY01
KFR26GW(26532)FNhCbA1
KFR26GW(26563)FNhAdB1JY01
KFR26GW(26575)FNhAaB3
KFR26GW(26553)FNhCbA1
KFR26GWNhHd1BG
KFR26GW(26565)FNhAaB1
KFR26GW(26519)FNhAaB3
KFR35GW(35575)FNhAaB3
KFR35GW(35563)FNhAdB1JY01
KFR35GWNhHd1BG
KFR35GW(35519)FNhAaB3
KFR26GW(26519)FNhBdB1
KFR26GW(26519)FNhBcB1
KFR26GW(26589)FNhAaB1
KFR35GW(35565)FNhAaB1
KFR35GW(35519)FNhBdB1
KFR35GW(35519)FNhBcB1
KFR26GW(26549)FNhAaB1
KFR50LW(50530)FNhAkB3
KFR35GW(35589)FNhAaB1
KFR26GW(26546)FNhAfB1
KFR26GW(26549)FNhCaB1
KFR50LWNhGh3B
KFR26GW(26537)FNhAaA1
KFR26GW(26537)FNhCaA1
KFR26GW(26546)FNhAdB1
KFR35GW(35549)FNhAaB1
KFR50LWNh

## 查出所有格力商品价格低于京东商品的条目

In [23]:
tmp_Str = 'KFR-35GW/\(35530\)FNhAk-B1'.replace('（','(').replace('）',')').replace('/','').replace('-','')
res = mycol.find({"$or":[{'名字':{ "$regex": tmp_Str}}]},{"_id":0}).sort("价格")
# res = mycol.find({"$or":[{'名字':{ "$regex": tmp_Str}}]},{"_id":0}).sort("价格")
res_gree = []
# for x in res:
#     patten = '([A-Za-z0-9]+\(?[A-Za-z0-9]*\)?[A-Za-z0-9]+)'
# #     print(x["名字"])
#     maxs = ""
#     for m in re.findall(patten,x["名字"]):
#         if len(maxs) < len(m):
#             maxs = m
#     res_gree.append(maxs)        
#     print(maxs)
drawpandas(res)


{'地址': 'https://mall.gree.com//goods/product/details?id=1002973198&skuId=1000302727', '产品匹数': '正1.5匹', '价格': 2899.0, '内机噪音dB（A）': '高风挡35', '内机尺寸（宽x高x深）mm': '825x293x196', '内机装箱清单': '室内机*1、遥控器*1、电池*2、说明书*2', '内机质量（kg）': '9.5', '制冷剂': 'R32', '制冷功率(W)': '810(75-2030)', '制冷量(W)': '3510(150-5210)', '制热功率(W)': '1250(90-2200)', '制热量(W)': '5010(150-6770)', '单冷/冷暖': '冷暖', '名字': '【新国标】天丽变频冷暖正1.5匹1级能效挂机空调 KFR35GW(35530)FNhAkB1(皓雪白)', '品牌': '格力', '外机噪音dB（A）': '51', '外机尺寸（宽x高x深）mm ': '802x555x350', '外机装箱清单': '室外机*1、连接管部件*1', '外机质量（kg）': '27.5', '定频/变频': '变频', '循环风量(m³/h)': '710', '挂机/柜机': '挂机', '是否电辅加热': '是', '电压/频率（V/Hz）': '220V～/50Hz', '电辅加热功率(W)': '1000', '能效等级': '新一级能效', '能源消耗效率': '5.27(全年能源消耗效率）', '适用面积（平方米）': '16-20', '颜色': '皓雪白'}


,综合系数,地址,价格,制冷剂,制冷功率(W),制热功率(W),能效等级,产品匹数,变频/定频
0,None,https://mall.gree.com//goods/product/details?i...,2899.0,R32,810(75-2030),1250(90-2200),新一级能效,正1.5匹,None


In [48]:
tmp = '【新国标】天井机3匹定频冷暖2级能效 380V电源 KFR72TW(72550S)NhCa2【新国标】天丽变频冷暖大1匹3级能效挂机空调 KFR26GW(26530)FNhAkB3(皓雪白)宁炫变频冷暖大1匹3级能效挂机空调 KFR26GWNhDdB3 皓雪白凉之夏III变频冷暖正1.5匹3级能效挂机空调 KFR35GW(35564)FNhAaC3 皓雪白悦风Ⅱ变频冷暖小1.5匹3级能效挂机空调 KFR32GW(32564)FNhAaA3皓雪白'
patten = '([A-Za-z0-9]+\(?[A-Za-z0-9]*\)?[A-Za-z0-9]+)'
print(re.findall(patten,tmp))

['380V', 'KFR72TW(72550S)NhCa2', 'KFR26GW(26530)FNhAkB3', 'KFR26GWNhDdB3', 'III', 'KFR35GW(35564)FNhAaC3', 'KFR32GW(32564)FNhAaA3']


## 保存优势价格列表
    


In [6]:
res_good = []
for good in res_gree:
    tmp_Str = good.replace('(','\(').replace(')','\)')
    res = mycol.find({"$or":[{'名字':{ "$regex": tmp_Str}},{'产品型号':{"$regex": tmp_Str}}]},{"_id":0}).sort([('价格',1),('地址',-1)])
    if good_in_list(res,'mall.gree.com','地址',0):
        tmp = {}
        res = mycol.find({"$or":[{'名字':{ "$regex": tmp_Str}},{'产品型号':{"$regex": tmp_Str}}]},{"_id":0}).sort([('价格',1),('地址',-1)])
        index = 0
        for tmp_good in res:
            if index == 1:
                tmp["京东价格"] = tmp_good['价格']
                tmp["京东地址"] = tmp_good['地址']
                break
            tmp["商品名称"] = tmp_good['名字']
            tmp["董店价格"] = tmp_good['价格']
            tmp["董店地址"] = tmp_good['地址']
            index = index + 1
        res_good.append(tmp)
headers = ['商品名称','董店价格','京东价格','董店地址','京东地址']
with open('result0613.csv','w',newline='',encoding='utf-8' )as f:
    f_csv = csv.DictWriter(f,headers)
    f_csv.writeheader()
    f_csv.writerows(res_good)
# print(len(res_good))
# print(res_good)


# drawpandas(res)

In [22]:
tmp_Str = 'KFR50LW(50553)FNhAbA3'.replace('(','\(').replace(')','\)')
res = mycol.find({"$or":[{'名字':{ "$regex": tmp_Str}},{'产品型号':{"$regex": tmp_Str}}]},{"_id":0}).sort('价格',1)
# print(type(res))
# print(good_in_list(res,'mall.gree.com','地址',5))
# res = mycol.find({"$or":[{'名字':{ "$regex": tmp_Str}},{'产品型号':{"$regex": tmp_Str}}]},{"_id":0}).sort('价格',1)
drawpandas(res)

{'地址': 'https://mall.gree.com//goods/product/details?id=1002979708&skuId=1000309564', '价格': 5999.0, '内机噪音dB（A）': '高风挡39', '内机尺寸（宽x高x深）mm': '402x1792x455', '内机装箱清单': '室内机*1、遥控器*1、电池*2、说明书*1', '内机质量（kg）': '40.5', '制冷剂': 'R32', '制冷功率(W)': '1620(255-2450)', '制冷量(W)': '5110(800-6250)', '制热功率(W)': '2300(255-2690)', '制热量(W)': '7150(850-7860)', '名字': 'i畅变频冷暖2匹3级能效柜机空调 KFR50LW(50553)FNhAbA3 (皓雪白)', '品牌': '格力', '外机噪音dB（A）': '≤54', '外机尺寸（宽x高x深）mm ': '899x596x378', '外机装箱清单': '室外机*1、连接管部件*1', '外机质量（kg）': '38', '循环风量(m³/h)': '1020', '是否电辅加热': '是', '电压/频率（V/Hz）': '220V～/50Hz', '电辅加热功率(W)': '2100', '能源消耗效率': '3.49(全年能源消耗效率）', '颜色': '皓雪白'}


,综合系数,地址,价格,制冷剂,制冷功率(W),制热功率(W),能效等级,产品匹数,变频/定频
0,None,https://mall.gree.com//goods/product/details?i...,5999.0,R32,1620(255-2450),2300(255-2690),None,None,None


In [4]:
res = mycol.find({},{"_id":0}).sort('价格',1)
index = 0
for i in res:
    index = index + 1
    print(i)
    if index == 10:
        break

{'地址': 'https://item.jd.com/100011872177.html', '产品匹数': '1.5匹', '产品型号': 'KFR35GWNhCf1BGj', '价格': 9.9, '优惠券': [], '内机净重(kg)': '12.5kg', '内机噪音dB（A）': '18-41', '内机尺寸（宽x高x深）mm': '929*330*184', '冷暖类型': '冷暖', '制冷剂': 'R32', '制冷功率(W)': '810（75-1800）', '制冷量(W)': '3510W', '制热功率(W)': '1260（90-2211）', '制热量(W)': '5010W', '名字': '<span class="ptag" style="backgroundcolor:#c81623">京品家电<span>\t格力(GREE)1.5匹 京淳 高效除甲醛 空调空气净化器一体机 壁挂式卧室空调挂机KFR35GWNhCf1BGj以旧换新', '外机净重(kg)': '29kg', '外机噪音dB（A）': '51', '外机尺寸（宽x高x深）mm': '802*555*350', '定频/变频': '变频', '循环风量(m³/h)': '650m3/h', '扫风方式': '上下/左右扫风', '智能APP远程操控': '品牌智能', '电压/频率（V/Hz）': '220V/50Hz', '电辅加热功率(W)': '1000W', '睡眠模式': '按键调节', '系列': '京淳系列', '能效等级': '新一级能效', '能源消耗效率': '5.28(APF)', '自动清洁': '内机自动清洁', '认证型号': 'KFR-35GW/NhCf1BGj', '质保年限': '十年质保', '质保说明': '整机十年包修', '面板材质': 'ABS塑料', '颜色': '蓝色'}
{'地址': 'https://item.jd.com/10030149044617.html', '价格': 79.0, '优惠券': ['免邮'], '名字': '格力空调原厂原装遥控器YAPOFB14 YAP0FB14 红色'}
{'地址': 'https://item.jd.com/100016964146.html', '产品匹数': '